# **Memory (RAG)**

The <samp>zyx</samp> library brings an easier way to interact with vector databases & implement RAG pipelines. Using the <code>Qdrant-Client</code> library, we can easily store and query data. The <code>completion</code> client has also been built into the Memory module to allow for LLM (RAG) completions.

## **Creating a Store**

Lets create two <code>Qdrant</code> collections to store and query data. The first one will hold string 'documents', the second will showcase the Pydantic store functionality.

In [1]:
import zyx
from pydantic import BaseModel


class Data(BaseModel):
    memory: str
    date: str


base_store = zyx.Memory(
    "base_store",
    location=":memory:",
    embedding_model="text-embedding-3-small",  # Defualt embedding model, can be changed to litellm compatible
    # embedding models.
)

pydantic_store = zyx.Memory(
    "pydantic_store",
    location=":memory:",
    model_class=Data,  # Set the Pydantic model class to use.
)

2024-09-08 00:33:13.825 | INFO     | zyx._client.memory:_create_collection:73 - Collection 'base_store' does not exist. Creating it now.
2024-09-08 00:33:13.826 | INFO     | zyx._client.memory:_create_collection:82 - Collection 'base_store' created successfully.
2024-09-08 00:33:13.827 | INFO     | zyx._client.memory:_create_collection:73 - Collection 'pydantic_store' does not exist. Creating it now.
2024-09-08 00:33:13.827 | INFO     | zyx._client.memory:_create_collection:82 - Collection 'pydantic_store' created successfully.


## **Adding Data**

In [2]:
class Memory(BaseModel):
    memory_and_date: str


base_data = zyx.generate(Memory, n=10)

print(base_data)

# Add it to the store
base_store.add(base_data)
base_store.add("My bestie and I went to the park yesterday 2023-08-01")

/home/hammad/miniconda3/envs/zyx/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


[
    Memory(memory_and_date='Visited the Grand Canyon on 2021-06-15'),
    Memory(memory_and_date='Graduated from college on 2019-05-20'),
    Memory(memory_and_date='First day at my new job on 2020-01-10'),
    Memory(memory_and_date='Family vacation to Hawaii on 2018-07-25'),
    Memory(memory_and_date='Learned to ride a bike on 2010-04-05'),
    Memory(memory_and_date="Attended my best friend's wedding on 2022-09-30"),
    Memory(memory_and_date='Moved into my first apartment on 2021-03-01'),
    Memory(memory_and_date='Saw my favorite band live on 2017-11-12'),
    Memory(memory_and_date='Celebrated my 30th birthday on 2023-02-18'),
    Memory(memory_and_date='Took a road trip across the country on 2020-08-15')
]

2024-09-08 00:33:19.423 | INFO     | zyx._client.memory:add:156 - Successfully added 10 points to the collection.
2024-09-08 00:33:19.616 | INFO     | zyx._client.memory:add:156 - Successfully added 1 points to the collection.


In [3]:
pydantic_data = zyx.generate(Data, n=10)

print(pydantic_data)

# Add it to the store
pydantic_store.add(pydantic_data)
pydantic_store.add(
    Data(memory="My bestie and I went to the park yesterday", date="2023-08-01")
)

[
    Data(memory='Remember to buy groceries', date='2023-10-01'),
    Data(memory="Doctor's appointment at 3 PM", date='2023-10-02'),
    Data(memory='Meeting with the project team', date='2023-10-03'),
    Data(memory='Call mom', date='2023-10-04'),
    Data(memory='Finish the report by Friday', date='2023-10-05'),
    Data(memory='Plan weekend trip', date='2023-10-06'),
    Data(memory='Attend yoga class', date='2023-10-07'),
    Data(memory='Submit tax documents', date='2023-10-08'),
    Data(memory='Grocery shopping list', date='2023-10-09'),
    Data(memory='Prepare for the presentation', date='2023-10-10')
]

2024-09-08 00:33:23.230 | INFO     | zyx._client.memory:add:156 - Successfully added 10 points to the collection.
2024-09-08 00:33:23.364 | INFO     | zyx._client.memory:add:156 - Successfully added 1 points to the collection.


## **Querying Data**

Lets query the data we just added.

In [4]:
results = base_store.search("Did i do anything with my best friend?")

print(results.query)
for result in results.results:
    print(result.text)

Did i do anything with my best friend?

My bestie and I went to the park yesterday 2023-08-01

{"memory_and_date": "Attended my best friend's wedding on 2022-09-30"}

{"memory_and_date": "Took a road trip across the country on 2020-08-15"}

{"memory_and_date": "Learned to ride a bike on 2010-04-05"}

{"memory_and_date": "Moved into my first apartment on 2021-03-01"}

In [5]:
results = pydantic_store.search("Did i do anything with my best friend?")

print(results.query)
for result in results.results:
    print(result.text)

Did i do anything with my best friend?

memory='My bestie and I went to the park yesterday' date='2023-08-01'

memory='Call mom' date='2023-10-04'

memory='Plan weekend trip' date='2023-10-06'

memory='Meeting with the project team' date='2023-10-03'

memory='Attend yoga class' date='2023-10-07'

## **LLM (RAG) Completions**

In [6]:
response = base_store.completion("Did i do anything with my best friend?")

print(response.choices[0].message.content)

2024-09-08 00:34:38.775 | INFO     | zyx._client.memory:completion:277 - Initial messages: Did i do anything with my best friend?


Yes, you went to the park with your best friend on August 1, 2023. Additionally, you attended your best friend's 
wedding on September 30, 2022.

In [7]:
response = pydantic_store.completion(
    "Did i do anything with my best friend?", model="ollama/llama3.1"
)

print(response.choices[0].message.content)

2024-09-08 00:34:39.951 | INFO     | zyx._client.memory:completion:277 - Initial messages: Did i do anything with my best friend?


Yes, according to your memory stored under ID: 80446fe4-acd5-4235-8552-65c01531f84f, you and your "bestie" (best 
friend) went to the park yesterday.